In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
""" Try to run everything
to make sure all works as should be
"""
import json
import pandas as pd
from tqdm import tqdm
import torch

import numpy as np
import umap
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from promptbook.similarity import SimilarityPlugin

/Users/alexander/opt/anaconda3/envs/winter/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def load_pl1l2(df_path="../datasets/subsamples_with_prompts.json"):
    with open(df_path, "r", encoding="utf-8") as file:
        df = json.load(file)

    df = pd.DataFrame(df)

    pl1l2 = df[df.parent_comment_text.notna()]

    del df
    return pl1l2

def build_embeddings(sp: SimilarityPlugin,
                     pl1l2: pd.DataFrame) -> tuple[torch.Tensor]:

    print("vectorize posts...")
    post_embeddings: torch.Tensor = sp.vectorizer(
        pl1l2["post_text"].values,
        tqdm_on=True
    )

    print("vectorize layer-1 comments...")
    layer1_embeddings: torch.Tensor = sp.vectorizer(
        pl1l2["parent_comment_text"].values,
        tqdm_on=True
    )

    print("vectorize layer-2 comments...")
    layer2_embeddings: torch.Tensor = sp.vectorizer(
        pl1l2["comment_text"].values,
        tqdm_on=True
    )

    return post_embeddings, layer1_embeddings, layer2_embeddings

In [4]:

sp = SimilarityPlugin(vectorizer_batch_size=50,
                        n_trees=100,
                        storage_path="/Users/alexander/Desktop/meow",
                        instance_name="test")

# Build embeddings
# __________________________________
pl1l2 = load_pl1l2()
post_embeddings, layer1_embeddings, layer2_embeddings = build_embeddings(sp, pl1l2)

# Result embedding := \sigma * P + \omega * L1 + \gamma * L2
SIGMA = 0.4
OMEGA = 0.2
GAMMA = 0.4
# embeddings should have shape (n, 768), where is number of objects
embeddings = SIGMA * post_embeddings + \
    OMEGA * layer1_embeddings + \
    GAMMA * layer2_embeddings

print(f"{embeddings.shape=}")

# Build storage
# __________________________________
payload: list = pl1l2.to_dict(orient="records")

train_size: int = int(0.55 * len(payload))
train_data = payload[:train_size]
val_data = payload[train_size:]

for idx, _ in enumerate(train_data):
    train_data[idx]["embedding"] = embeddings[idx]

print("Building the storage...")
sp.build_storages(train_data)
print("The storage has been build")

vectorize posts...


100%|██████████| 1/1 [01:11<00:00, 71.23s/it]


vectorize layer-1 comments...


100%|██████████| 1/1 [00:13<00:00, 13.93s/it]


vectorize layer-2 comments...


100%|██████████| 1/1 [00:06<00:00,  6.35s/it]


embeddings.shape=(47, 768)
Building the storage...
The storage has been build


In [5]:
SIGMA = 0.4
OMEGA = 0.2
GAMMA = 0.4

In [6]:
sp.return_format_fn = lambda x: (x["post_text"],
                                 x["comment_text"],
                                 x["parent_comment_text"])

def vectorizing_strategy(vectorizer, data):
    print(f"vectorizing_strategy    {data}")
    embds = []
    for row in tqdm(data):
        post_embd: str = vectorizer(row["post_text"])
        layer1_embd: str = vectorizer(row["parent_comment_text"])
        layer2_embd: str  = vectorizer(row["comment_text"])

        embd = SIGMA * post_embd + OMEGA * layer1_embd + GAMMA * layer2_embd
        embds.append(embd)

    return embds

In [16]:
simlr = sp.process(val_data,
                   vectorizing_strategy=vectorizing_strategy,
                   strategy="centroid_outsider",
                   strategy_object=None)

# embds = torch.cat(embds)

vectorizing_strategy    [{'Unnamed: 0': 51025, 'mention_id': 15648403, 'page_id': 38576077, 'post_id': 1520225, 'post_text': '🆕 Пенсионерка с Будапештской поверила в россказни лжесотрудницы Сбера и набрала кредитов на 250 тысяч рублей 15 марта в полицию обратилась 71-летняя пенсионерка, проживающая на улице Будапештской. Петербурженка рассказала, что 13 марта на ее мобильный телефон позвонила неизвестная женщина, которая представилась сотрудником ПАО «Сбербанк». Злоумышленница убедила жертву в том, что от ее имени неизвестные оформили договора на получение кредитов. Для предотвращения преступных действий пенсионерка должна была сама оформить кредит и все деньги перевести на безопасный счет. Петербурженка поверила голосу из телефона и, перепугавшись, оформила кредит на 250 тысяч рублей в банке на Малой Балканской. Взяв деньги, женщина пошла в продуктовый магазин, где в банкомате перевела их на неизвестные счета. О том, что стала жертвой обмана, пенсионерка поняла лишь спустя два дня. Ис

100%|██████████| 22/22 [00:05<00:00,  4.05it/s]

[{'Unnamed: 0': 180408, 'mention_id': 17204238, 'page_id': 59270452, 'post_id': 10720406, 'post_text': 'Часто слышу историй о том , как мошенники набирают с 900 , типа работники Сбера и раскручивают людей на бабки. Кто-то сталкивался лично?', 'comment_id': 10720559, 'user_id': 723508107, 'comment_text': 'пока неизвестно 😅', 'parent_comment_id': 10720510.0, 'parent_synth_id': 'u_723508107-c_10720510', 'parent_comment_text': 'У меня эффект дежавю,где то я это все уже видела', 'isnan': False, 'later': False, 'synth_comment_id': 'u_723508107-c_10720559', 'prompts': '{"id": "u_723508107-c_10720559", "A": "Часто слышу историй о том , как мошенники набирают с 900 , типа работники Сбера и раскручивают людей на бабки. Кто-то сталкивался лично?", "B": "У меня эффект дежавю,где то я это все уже видела", "C": "пока неизвестно 😅"}'}, {'Unnamed: 0': 57059, 'mention_id': 21627943, 'page_id': 213387262, 'post_id': 106, 'post_text': 'СБЕРБАНК СНИЖАЕТ СТАВКИ – ДО КОНЦА ЛЕТА ИПОТЕКА НА САМЫХ ВЫГОДНЫХ УСЛ

In [17]:
simlr

[('Сегодня 31.03.23 на дрезденской улице Выборгский район найдена карта Сбер МИР на имя Екатерина Васильева',
  'спасибо! Отправила. О результате сообщу.',
  'Лайфхак ,Берете номер карты и отправляете рубль с пометкой ,что нашли карту !(Мне так вернули мою как то)'),
 ('✅ Специалисты ПАО «Сбербанк» провели урок финансовой грамотности для сотрудников КДЦ «Южный» Г.о. Подольск. \u2063\u2063⠀ \u2063\u2063⠀ 👨\u200d💼 Прошедшая встреча приурочена к проведенному в марте этого года межрегиональному фестивалю финансовой культуры и грамотности. На мероприятии представители ПАО «Сбербанк» напомнили про возможность снятия наличных денежных средств с карты и оплаты услуг ЖКХ на кассах продуктовых магазинов округа. Теперь это можно сделать в магазинах сети «Перекресток», «Верный», «Вкусвилл» и во всех магазинах сети «Пятёрочка». Помимо прочего, они объяснили правила действий при поступлении телефонных звонков от мошенников.\u2063\u2063⠀ \u2063\u2063⠀ 🛡 Помимо этого, специалисты проинформировали сотр

In [290]:
# reducer = umap.UMAP(n_components=3, random_state=42)
# embds_reduced = reducer.fit_transform(embds)

# # Convert reduced embeddings from polar (spherical) coordinates to Cartesian coordinates for plotting
# # Note: The output of Haversine metric is in latitude and longitude, which needs to be converted
# x = np.cos(embds_reduced[:, 0]) * np.cos(embds_reduced[:, 1])
# y = np.sin(embds_reduced[:, 0]) * np.cos(embds_reduced[:, 1])
# z = np.sin(embds_reduced[:, 1])

# Фиксируем результат:

Эвристически, лучшие коэффициенты для получения итогового векторного представления треда суть:

$$\Sigma = 0.4 \\ \Omega = 0.2 \\ \Gamma = 0.4$$

$$ \mathcal{V} = \Sigma * \text{post} + \Omega * \text{L1} + \Gamma * \text{L2}$$

# Centroid & Outsider

In [317]:
class Strategy:
    def __init__(self,
                 embeddings: torch.Tensor) -> None:
        self.embeddings = embeddings

    def run(self):
        """ Choose embeddings
        """
        raise NotImplemented

In [324]:
class CentroidAndOutsiderStrategy(Strategy):
    def __init__(self, embeddings: torch.Tensor):
        super().__init__(embeddings)

    def run(self):
        return self.calculate_anchor_points()

    def calculate_anchor_points(self) -> dict[str, int]:
        centroid_idx: int = 0
        outsider_idx: int = 0
        current_centroid_delta: int = 1e8
        current_outsider_delta: int = 0
        center_of_mass: torch.Tensor = self.embeddings.mean(dim=0)

        # euclidian
        metric: callable = lambda a, b: torch.sqrt(torch.sum((a-b)**2))

        for idx, emb in enumerate(self.embeddings):
            if (delta := metric(emb, center_of_mass)) < current_centroid_delta:
                current_centroid_delta = delta
                centroid_idx = idx

            if (delta := metric(emb, center_of_mass)) > current_outsider_delta:
                current_outsider_delta = delta
                outsider_idx = idx
        
        return {"centroid_idx": centroid_idx, "outsider_idx": outsider_idx}

In [325]:
cao = CentroidAndOutsiderStrategy(embds)
cao.run()

{'centroid_idx': 9, 'outsider_idx': 17}

In [301]:
def calculate_anchor_points(embeddings: torch.Tensor) -> dict[str, int]:
    centroid_idx: int = 0
    outsider_idx: int = 0
    current_centroid_delta: int = 1e8
    current_outsider_delta: int = 0
    center_of_mass: torch.Tensor = embeddings.mean(dim=0)

    # euclidian
    metric: callable = lambda a, b: torch.sqrt(torch.sum((a-b)**2))

    for idx, emb in enumerate(embeddings):
        if (delta := metric(emb, center_of_mass)) < current_centroid_delta:
            current_centroid_delta = delta
            centroid_idx = idx

        if (delta := metric(emb, center_of_mass)) > current_outsider_delta:
            current_outsider_delta = delta
            outsider_idx = idx
    
    return {"centroid_idx": centroid_idx, "outsider_idx": outsider_idx}

In [304]:
reducer = umap.UMAP(n_components=2, random_state=42, n_jobs=1)
embds_reduced = reducer.fit_transform(embds)
embds_reduced = torch.from_numpy(embds_reduced)

In [305]:
centroid_idx, outsider_idx = calculate_anchor_points(embds_reduced).values()

In [ ]:
view_data = pd.DataFrame(embds_reduced)

view_data["labels"] = labels

for label in np.unique(labels):
    # Select data points belonging to the current label/cluster
    cluster_data = view_data[view_data.labels == label]
    
    # Add scatter plot for this cluster
    fig.add_trace(go.Scatter3d(
        x=cluster_data[0],  # X-axis data
        y=cluster_data[1],  # Y-axis data
        z=cluster_data[2],  # Z-axis data
        mode='markers',  # Plot type
        marker=dict(
            size=5,  # Marker size
            opacity=0.8,  # Marker opacity
            color="red"
        ),
        name=f'Cluster {label}'  # Legend label
    ))

# Update the layout for a better visual
# fig.update_layout(
#     title='Clustered Data Points in 3D',
#     scene=dict(
#         xaxis_title='X',
#         yaxis_title='Y',
#         zaxis_title='Z'
#     )
# )

# Show the plot
fig.show()

# Clustering

In [184]:
from sklearn.cluster import DBSCAN


def dbscan_clustering(data, eps=1e1, min_samples=2, metric="cosine"):
    """
    Apply DBSCAN clustering algorithm to data.

    Parameters:
    - data: NumPy array of shape (n_examples, n_features)
    - eps: The maximum distance between two samples for them to be considered as in the same neighborhood.
    - min_samples: The number of samples in a neighborhood for a point to be considered as a core point.

    Returns:
    - labels: Cluster labels for each point. Noisy samples are given the label -1.
    """
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(data)
    labels = db.labels_
    return labels, db

In [194]:
from sklearn.cluster import HDBSCAN

def hdbscan_clustering(data, min_cluster_size=5, min_samples=None):
    """
    Apply HDBSCAN clustering algorithm to data.

    Parameters:
    - data: NumPy array of shape (n_examples, n_features)
    - min_cluster_size: The minimum size of clusters; otherwise, the cluster is considered as noise.
    - min_samples: The number of samples in a neighborhood for a point to be considered as a core point.

    Returns:
    - labels: Cluster labels for each point. Noisy samples are given the label -1.
    """
    clusterer = HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples)
    clusterer.fit(data)
    labels = clusterer.labels_
    return labels

# Example usage
labels = hdbscan_clustering(embds_reduced, min_cluster_size=2, min_samples=1)

print(labels)

[ 3  2  6  6  3  7  5  5 -1  4  8  1  0  5  8  2  0 -1  1  7  4  0]


In [201]:
view_data[view_data.labels == label]

,0,1,2,labels
8,7.204446,-5.111145,8.073632,-1
17,6.455878,-5.408100,8.614894,-1


In [204]:
view_data[view_data.labels == label][0]

8     7.204446
17    6.455878
Name: 0, dtype: float32

In [211]:
view_data = pd.DataFrame(embds_reduced)

view_data["labels"] = labels

for label in np.unique(labels):
    # Select data points belonging to the current label/cluster
    cluster_data = view_data[view_data.labels == label]
    
    # Add scatter plot for this cluster
    fig.add_trace(go.Scatter3d(
        x=cluster_data[0],  # X-axis data
        y=cluster_data[1],  # Y-axis data
        z=cluster_data[2],  # Z-axis data
        mode='markers',  # Plot type
        marker=dict(
            size=5,  # Marker size
            opacity=0.8,  # Marker opacity
            color="red"
        ),
        name=f'Cluster {label}'  # Legend label
    ))

# Update the layout for a better visual
# fig.update_layout(
#     title='Clustered Data Points in 3D',
#     scene=dict(
#         xaxis_title='X',
#         yaxis_title='Y',
#         zaxis_title='Z'
#     )
# )

# Show the plot
fig.show()

In [212]:
labels

array([ 3,  2,  6,  6,  3,  7,  5,  5, -1,  4,  8,  1,  0,  5,  8,  2,  0,
       -1,  1,  7,  4,  0])

In [197]:
c = {}
for i in labels:
    if i in c:
        c[i] += 1
    else:
        c[i] = 1 

c

{3: 2, 2: 2, 6: 2, 7: 2, 5: 3, -1: 2, 4: 2, 8: 2, 1: 2, 0: 3}

In [180]:
labels, db_ = dbscan_clustering(embds_reduced)

In [181]:
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [178]:
db_.metric


'euclidean'